In [1]:
import re
import csv
import pandas as pd
import os
from camel_tools.utils import charsets
import sys
# insert at 1, 0 is the script path (or '' in REPL)
sys.path.append('/Users/nurpeiis/Desktop/Capstone/hierarchical-did/utils')
from data_process import DataProcess

In [9]:
def get_speakers_info():
    df= pd.read_csv('../../data_raw/ldc_arb_gulf_cttr_2006_t15/doc/training_calls.tab', delimiter='\t') 
    df_speaker_info = pd.DataFrame()
    df_speaker_info['filename'] = df['File-ID']    
    df_speaker_info['A'] = df['A-Region']  
    df_speaker_info['B'] = df['B-Region']
    df_devtest= pd.read_csv('../../data_raw/ldc_arb_gulf_cttr_2006_t15/doc/devtest_calls.tab', delimiter='\t') #in line 4 of the file delete extra tabs from Places to go
    df_speaker_info_devtest = pd.DataFrame()
    df_speaker_info_devtest['filename'] = df_devtest['File-ID']    
    df_speaker_info_devtest['A'] = df_devtest['A-Region']  
    df_speaker_info_devtest['B'] = df_devtest['B-Region']
    df_speaker_info = df_speaker_info.append(df_speaker_info_devtest, ignore_index=True)
    return df_speaker_info

,filename,A,B
0,arb_glf-20040208_011139,UAE,UAE
1,arb_glf-20040208_012423,Oman,Oman
2,arb_glf-20040208_013620,UAE,UAE
3,arb_glf-20040208_015029,UAE,UAE
4,arb_glf-20040208_015804,Oman,Oman
...,...,...,...
521,arb_glf-20040301_041731,Oman,Oman
522,arb_glf-20040304_052938,Saudi_Arabia,Saudi_Arabia
523,arb_glf-20040310_075915,Saudi_Arabia,Saudi_Arabia
524,arb_glf-20040315_061838,Oman,Oman


In [20]:
def speakers_split(path_to_directory, filename):
    # Cleaning data and splitting between speaker A and B
    with open('{}/{}'.format(path_to_directory, filename)) as f:
        try:
            lines_raw = f.read().splitlines()
        except:
            return False
        lines = [i for i in lines_raw if i] 
    ar_letters = charsets.AR_LETTERS_CHARSET
    reg=re.compile('^[{}]+$'.format(ar_letters))
    speakers = {'A': [], 'B': []}
    curr_speaker = 'A'
    for l in lines:
        word = l.split()
        line = ""
        for w in word:
            if w == 'A:' or w == 'B:' or reg.match(w):
                line += w + " "
        line = line[:-1]
        if 'A:' in line:
            curr_speaker = 'A'
            line = line.replace('A:','')
        elif 'B:' in line:
            curr_speaker = 'B'            
            line = line.replace('B:','')
        line = line.replace('tnfs', '')
        #print(line)
        #Get only lines that are purely in Arabic
        line = line.replace('(', '')        
        line = line.replace(')', '')
        #print(line)
        if re.match("[\(A-Za-z]", line) == None and line != '':
            speakers[curr_speaker].append(line)
    return speakers
#speakers_split('../../data_raw/ldc_arb_gulf_cttr_2006_t15/transc/train2c/', 'arb_glf-20040208_011139.txt')

In [14]:
def get_processed(speakers, country_a, country_b):
    df_processed = pd.DataFrame()
    df_processed['original_sentence'] = speakers['A']
    df_processed['dialect_country_id'] = country_a
    df_processed['dialect_region_id'] = 'Gulf'
    df_processed_b = pd.DataFrame()
    df_processed_b['original_sentence'] = speakers['B']
    df_processed_b['dialect_country_id'] = country_b
    df_processed_b['dialect_region_id'] = 'Gulf'

    df_processed = df_processed.append(df_processed_b, ignore_index=True)
    return df_processed

In [18]:
def process_directory(path_to_directory):
    df = pd.DataFrame(columns={'original_sentence', 'dialect_country_id', 'dialect_region_id'})

    df_speakers = get_speakers_info()
    counter_bad = 0
    files = os.listdir(path_to_directory)
    for f in files:
        try:
            country_a = df_speakers.loc[df_speakers["filename"] == f[:-4]].iloc[0]['A']
            country_b = df_speakers.loc[df_speakers["filename"] == f[:-4]].iloc[0]['B']
            speakers = speakers_split(path_to_directory, f)
            if speakers:
                df = df.append(get_processed(speakers, country_a, country_b), ignore_index=True)
            else:
                counter_bad += 1
        except:
            print("Bad", f)
            continue
    print('Unable to process {} files in the following directory: {}'.format(counter_bad, path_to_directory))
    return df

In [23]:
df_train = process_directory('../../data_raw/ldc_arb_gulf_cttr_2006_t15/transc/train2c')
df_train.to_csv('../../data_raw/ldc_arb_gulf_cttr_2006_t15/train_processed.tsv', sep='\t', index=False)

Unable to process 0 files in the following directory: ../../data_raw/ldc_arb_gulf_cttr_2006_t15/transc/train2c


In [31]:
df_devtest = process_directory('../../data_raw/ldc_arb_gulf_cttr_2006_t15/transc/devtest')
df_devtest.to_csv('../../data_raw/ldc_arb_gulf_cttr_2006_t15/devtest_processed.tsv', sep='\t', index=False)

Unable to process 0 files in the following directory: ../../data_raw/ldc_arb_gulf_cttr_2006_t15/transc/devtest


In [2]:
dp = DataProcess('../data_processed/ldc_arb_gulf_cttr_2006_t15/', 'user_level', 'speech_transcript', 'https://catalog.ldc.upenn.edu/LDC2006T15', 'ldc_arb_gulf_cttr_2006_t15', {},{},2, 'corpus', 'original')

In [29]:
dp.save_file('train_processed.tsv', dp.preprocess('../../data_raw/ldc_arb_gulf_cttr_2006_t15/train_processed.tsv', '', '', 1, 0, header=0))
dp.save_file('devtest_processed.tsv', dp.preprocess('../../data_raw/ldc_arb_gulf_cttr_2006_t15/devtest_processed.tsv', '', '', 1, 0, header=0))

In [8]:
df_train = pd.read_csv('../data_processed/ldc_arb_gulf_cttr_2006_t15/train_processed.tsv', delimiter='\t', header=0)
df_train['split_original_manual'] = 'original'
df_test = pd.read_csv('../data_processed/ldc_arb_gulf_cttr_2006_t15/devtest_processed.tsv', delimiter='\t', header=0)
df_test['split_original_manual'] = 'original'
df_train.to_csv('../data_processed_splited/ldc_arb_gulf_cttr_2006_t15/train_processed.tsv', sep='\t', index=False)
df_test.to_csv('../data_processed_splited/ldc_arb_gulf_cttr_2006_t15/devtest_processed.tsv', sep='\t',index=False)





In [9]:
dp.save_features('../datasets_splited_features.tsv')